In [183]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import CosineRecommender
from sklearn.preprocessing import minmax_scale
import os
os.environ['OPENBLAS_NUM_THREADS'] = '1'
os.environ['MKL_NUM_THREADS'] = '1'

In [184]:
df = pd.read_csv('data_ml-1m/ratings.csv')

In [185]:
df

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291
...,...,...,...,...
1000204,6040,1091,1,956716541
1000205,6040,1094,5,956704887
1000206,6040,562,5,956704746
1000207,6040,1096,4,956715648


# Создаем разреженную матрицу

In [186]:
user_item_matrix = coo_matrix((
    (df["rating"]>=4).astype(np.float32),
    (df["userId"], df["movieId"])))

user_item_matrix.eliminate_zeros()

# Разделение на обучающую и тестовые выборки

In [187]:
total_len = user_item_matrix.data.size
train_len = int(total_len * 0.8)
all_indices = np.arange(total_len)
np.random.seed(42)
train_indices = np.random.choice(all_indices, train_len, replace = False)
train_mask = np.in1d(all_indices, train_indices)

# Функция наложения маски

In [188]:
def get_masked(arr, mask):
    return coo_matrix(
        (
            [np.float32(item) for item in arr.data[mask]],
            (arr.row[mask], arr.col[mask])
        ), arr.shape
    )

# Функция получения рекомендаций

In [189]:
def get_recs(user, model):
    return {
        user: model.recommend(userid=user, user_items=train_csr, N=50)
        for user in users
    }

# Хитрейт 

In [190]:
def hitrate (k, recs, users):
    hits = 0
    for user in users:
        if recs[user]:
            rec_items, _ = zip(*recs[user])
            hits += len(set(rec_items[:k]).intersection(set(test_csr[user].indices))) > 0
    return hits / len(users) 

# Подготовка матриц

In [191]:
train_csr = get_masked(user_item_matrix, train_mask).tocsr()
train = train_csr.T
test_coo = get_masked(user_item_matrix, ~train_mask)
test_csr = test_coo.tocsr()

In [192]:
users = list(set(test_coo.row))

# ALS и Cosine отдельно

In [193]:
als = AlternatingLeastSquares(random_state = 42)
als.fit(train)
recs_als = get_recs(users, als)
cos = CosineRecommender()
cos.fit(train)
recs_cosine = get_recs(users, cos)

# Функция нормализации данных 

In [194]:
def normalize(alg, users):
    for user in users:
        if alg[user]:
            rec_items, rec_users = zip(*alg[user])
            rec_users = list(minmax_scale(list(rec_users)))
            for i in range(len(alg[user])):
                alg[user][i] = (rec_items[i], rec_users[i])
    return alg
recs_als_norm = normalize(recs_als, users)
recs_cos_norm = normalize(recs_cosine, users)

# Смешивание

In [195]:
def mixed_recs(recs_als, recs_cos, weight=1):
    mixed_recs = dict()
    for i in recs_als:
        mixed_recs[i] = list()
        dict_recs_als = dict(recs_als[i])
        dict_recs_cos = dict(recs_cos[i])
        set_keys = set(list(dict_recs_als.keys()) + list(dict_recs_cos.keys()))
        for j in set_keys:
            if dict_recs_als.get(j) != None and dict_recs_cos.get(j) != None:
                mixed_recs[i].append((j, (dict_recs_als.get(j))*weight + (dict_recs_cos.get(j))*(1-weight)))
            elif dict_recs_als.get(j) != None:
                mixed_recs[i].append((j, (dict_recs_als.get(j))*weight))
            elif dict_recs_cos.get(j) != None:
                mixed_recs[i].append((j, (dict_recs_cos.get(j))*weight))
            else: 
                mixed_recs[i].append(j, 0)
    top_mixed = dict()
    for i in mixed_recs:
        top_mixed[i] = sorted(mixed_recs[i], key = lambda x: x[1], reverse = True)[:50]
    return top_mixed
recs_sum = mixed_recs(recs_als_norm, recs_cos_norm, 0.7)

In [196]:
print('als', 'hitrate=50  ', hitrate(50, recs_als, users))
print('cos', 'hitrate=50  ', hitrate(50, recs_cosine, users))
print('mix', 'hitrate=50  ', hitrate(50, recs_sum, users))

als hitrate=50   0.9280936454849499
cos hitrate=50   0.9349498327759197
mix hitrate=50   0.9461538461538461


# За опоздание с дедлайном штраф анекдот

Я регулярно захожу в пропускной пункт на границе с Украиной, и каждый раз спрашиваю, можно ли пройти через границу. Каждый раз мне отвечают "нет". Я спрашивал уже раз 150 и 150 раз мне ответили нет. Смысл в том, что отвечает мне один и тот же проверяющий, отвечает с неизменной интонацией. А я каждый раз с неизменной интонацией спрашиваю:
- Пройти можно?
- У вас паспорта нет.
- Украина - великая страна, паспорт не нужен.

- Можно мне пройти через границу?
- Извините, у вас паспорт поддельный.
- Мой паспорт вам не нравится, я понял. Я потом приду, новый будет лучше!

- Привет старина, это опять я.
- Нужно разрешение на въезд.

- Я дам тебе 10 кредитов за правильный штампик.
- Извините, но пропуск недействителен.

- Слава Украини, можно пройти?
- Извините но вес не совпадает, вы провозите что-нибудь с собой.

И ведь этот пограничник, зараза, знает меня идеально в лицо, знает, что я спрошу и знает, что он мне ответит. Но ещё ни разу ни один из нас ни жестом, ни словом не показал, что каждый из нас знает сценарий. Бывает проверяющий смотрит фотографию своей семьи, когдя я захожу в будку, тогда я подсматриваю за ним сквозь стеклянное окошко, он равнодушно досматривает фотографию, кладет её в карман и возвращается за будку:
- Что вы хотели?
- Дружище, мне бы пройти за границу.
- У вас документов нет.
- Жаль.

Иногда он общается с другим иммигрантом, тогда, стоя рядом, я терпеливо жду, когда они закончат:
- А мне бы пройти через границу?
- Нет, у вас документы просроченны.

Иногда он просто скучает за столом, когда в пропускном пункте никого нет, и только я расхаживаю по залу. Конечно, он знает, что будет дальше, но не подаёт виду и равнодушно берёт журнал.
- Привет дружище, пройти можно?
- Нет, с контрабандой нельзя.

Это очень суровое, по-настоящему мужское противостояние, исход которого не ясен. Очевидно, что каждая сторона рассчитывает на победу. Впрочем, я уже согласен на ничью.